In [1]:
import tableauserverclient as TSC
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:
e = load_dotenv(".tableau_env")

In [3]:
TABLEAU_USERNAME = os.environ.get("TABLEAU_USERNAME")
TABLEAU_PASSWORD = os.environ.get("TABLEAU_PASSWORD")
TABLEAU_SITENAME = os.environ.get("TABLEAU_SITENAME")
TABLEAU_SERVER_URL = os.environ.get("TABLEAU_SERVER_URL")
# for auth via token
TABLEAU_TOKEN_NAME = os.environ.get("TABLEAU_TOKEN_NAME")
TABLEAU_TOKEN_VALUE = os.environ.get("TABLEAU_TOKEN_VALUE")

In [4]:
# connect to tableau server
tableau_auth = TSC.PersonalAccessTokenAuth(TABLEAU_TOKEN_NAME, TABLEAU_TOKEN_VALUE, TABLEAU_SITENAME)
server = TSC.Server(f"https://{TABLEAU_SERVER_URL}", use_server_version=True)
server.auth.sign_in(tableau_auth)

#### Data Sources - not used for now

In [5]:
# get all datasources and their ID => ideally this is an ingestion job
# and the data sources are gonna be saved in a table
all_datasources, pagination_item = server.datasources.get()

# to make it easy, just save the data sources as dict, alongside their ID
datasources_dict = {i.name: i.id for i in all_datasources}

In [6]:
# get the data source to refresh by ID
datasource = server.datasources.get_by_id(datasources_dict["Alarm RLS"])

# call the refresh method with the data source item
refreshed_datasource = server.datasources.refresh(datasource)

#### Workbooks

In [7]:
# get all workbooks and their ID => ideally this is an ingestion job
# and the workbooks are gonna be saved in a table
# we also need projects
all_workbooks_items, pagination_item = server.workbooks.get()
all_project_items, pagination_item = server.projects.get()

# unpack payload in dict
#workbooks_dict = {i.name: i.id for i in all_workbooks_items}


In [8]:
# these two dataframes are gonna be tables in snowflake
df_projects = pd.DataFrame([(i.id, i.parent_id, i.name) for i in all_project_items], columns=["project_id", "parent_project_id", "project_name"])

df_workbooks = pd.DataFrame(
    [(i.content_url, i.created_at, i.id, i.name, i.owner_id, i.project_id, i.project_name, i.size, i.show_tabs, i.hidden_views, i.tags, i.updated_at, i.webpage_url) for i in all_workbooks_items],
    columns=["content_url", "created_at", "id", "name", "owner_id", "project_id", "project_name", "size", "show_tabs", "hidden_views", "tags", "updated_at", "webpage_url"]
)

In [13]:
df_workbooks.head()

,content_url,created_at,id,name,owner_id,project_id,project_name,size,show_tabs,hidden_views,tags,updated_at,webpage_url
0,Regional,2022-10-21 07:12:04+00:00,116426e6-11b4-4438-babd-1110d3df3554,Regional,af745434-6e79-4655-bb53-af3a3e427582,fbf35beb-743d-4f44-b508-e461f6906d73,Samples,2,True,None,{},2022-10-21 07:12:04+00:00,https://dub01.online.tableau.com/#/site/factor...
1,Superstore,2022-10-21 07:12:09+00:00,54135fc8-60ea-490d-aacf-52c0275c95d8,Superstore,af745434-6e79-4655-bb53-af3a3e427582,fbf35beb-743d-4f44-b508-e461f6906d73,Samples,1,True,None,{},2022-10-21 07:12:09+00:00,https://dub01.online.tableau.com/#/site/factor...
2,AdminInsightsStarter,2022-10-23 07:33:13+00:00,aa0bd6e8-6964-4da6-b684-4e85179b8af7,Admin Insights Starter,e2bc92c6-2f4e-40db-9755-8ac94c2bc4f0,baee820c-602a-496c-9753-8fb61007851a,Admin Insights,2,True,None,{},2023-01-15 08:02:39+00:00,https://dub01.online.tableau.com/#/site/factor...
3,Snowflakemonitoring,2022-11-01 13:13:04+00:00,5225bbd6-ffdd-49ba-a905-47ac53903c38,Snowflake monitoring,263efb4c-f589-4e11-9020-c11cf5a7b871,fa55702c-b95c-4568-8a40-afe280323bd1,Insight,1,False,None,{},2023-12-10 09:31:24+00:00,https://dub01.online.tableau.com/#/site/factor...
4,DowntimeAnalysis,2022-11-15 14:19:31+00:00,1a89cd94-864f-43a5-9448-d5b17ddf290d,Downtime Analysis,c1676c99-b99d-49cd-bc16-ab73fad840f0,fa55702c-b95c-4568-8a40-afe280323bd1,Insight,2,False,None,{},2023-02-28 07:34:47+00:00,https://dub01.online.tableau.com/#/site/factor...


##### This uses only Tableau metadata for triggering refreshes by specifying one or more projects of interest (can be done through a seed file)

In [9]:
# here we apply some filters to get to the ID of the projects to refresh (basically Operation => Certified folder)
# this assumes a fixed structured; if this changes then we are not so flexible
proj_to_target = df_projects[df_projects["project_name"] == 'Operation'].project_id.values[0]
sub_proj_to_target = df_projects[(df_projects["parent_project_id"] == proj_to_target) & (df_projects["project_name"] == "3. Certified")].project_id.values[0]
project_to_refresh = df_projects[df_projects["parent_project_id"] == sub_proj_to_target].project_id.values

In [10]:
project_to_refresh

array(['8c88e8fe-bce7-4258-88c3-3848ea6e6250',
       '8c1d88a2-c6b8-4c07-a1d0-fe80dd01f6d2'], dtype=object)

In [11]:
# here we get the workbook to refresh
workbooks_to_refresh = df_workbooks[df_workbooks["project_id"].isin(project_to_refresh)].id.values

In [12]:
workbooks_to_refresh

array(['d7109010-4922-4074-a993-1e21d00289e4',
       'a45b43c4-dd52-4cb2-95b9-69adcb142805',
       '13c98795-fe43-4660-8b8c-baa51e40def4',
       '66ec9509-a441-401c-80f3-043598a0b327',
       'dbc2c8e9-0e92-4eb2-8caf-bfcede45c858',
       '275126a2-f05e-4f1d-b38d-a06fead2f421'], dtype=object)

In [99]:
# loops through workbooks to refresh and trigger refresh
# run only if you want to refresh the specific dashboard
for w in workbooks_to_refresh:
    workbook = server.workbooks.get_by_id(w)
    # call the refresh method with the data source item
    refreshed_workbook = server.workbooks.refresh(workbook)
    print(f"refresh for workbook id {w} triggered")

##### This uses only Tableau metadata for triggering refreshes by specifying a specific tag (mantained through Tableau)

In [30]:
# get all workbooks and their ID => ideally this is an ingestion job
# and the workbooks are gonna be saved in a table
# here if we use the tag we don't need the project info
all_workbooks_items, pagination_item = server.workbooks.get()

df_workbooks = pd.DataFrame(
    [(i.content_url, i.created_at, i.id, i.name, i.owner_id, i.project_id, i.project_name, i.size, i.show_tabs, i.hidden_views, i.tags, i.updated_at, i.webpage_url) for i in all_workbooks_items],
    columns=["content_url", "created_at", "id", "name", "owner_id", "project_id", "project_name", "size", "show_tabs", "hidden_views", "tags", "updated_at", "webpage_url"]
)

In [17]:
df_workbooks.head()

,content_url,created_at,id,name,owner_id,project_id,project_name,size,show_tabs,hidden_views,tags,updated_at,webpage_url
0,Regional,2022-10-21 07:12:04+00:00,116426e6-11b4-4438-babd-1110d3df3554,Regional,af745434-6e79-4655-bb53-af3a3e427582,fbf35beb-743d-4f44-b508-e461f6906d73,Samples,2,True,None,{},2022-10-21 07:12:04+00:00,https://dub01.online.tableau.com/#/site/factor...
1,Superstore,2022-10-21 07:12:09+00:00,54135fc8-60ea-490d-aacf-52c0275c95d8,Superstore,af745434-6e79-4655-bb53-af3a3e427582,fbf35beb-743d-4f44-b508-e461f6906d73,Samples,1,True,None,{},2022-10-21 07:12:09+00:00,https://dub01.online.tableau.com/#/site/factor...
2,AdminInsightsStarter,2022-10-23 07:33:13+00:00,aa0bd6e8-6964-4da6-b684-4e85179b8af7,Admin Insights Starter,e2bc92c6-2f4e-40db-9755-8ac94c2bc4f0,baee820c-602a-496c-9753-8fb61007851a,Admin Insights,2,True,None,{},2023-01-15 08:02:39+00:00,https://dub01.online.tableau.com/#/site/factor...
3,Snowflakemonitoring,2022-11-01 13:13:04+00:00,5225bbd6-ffdd-49ba-a905-47ac53903c38,Snowflake monitoring,263efb4c-f589-4e11-9020-c11cf5a7b871,fa55702c-b95c-4568-8a40-afe280323bd1,Insight,1,False,None,{},2023-12-10 09:31:24+00:00,https://dub01.online.tableau.com/#/site/factor...
4,DowntimeAnalysis,2022-11-15 14:19:31+00:00,1a89cd94-864f-43a5-9448-d5b17ddf290d,Downtime Analysis,c1676c99-b99d-49cd-bc16-ab73fad840f0,fa55702c-b95c-4568-8a40-afe280323bd1,Insight,2,False,None,{},2023-02-28 07:34:47+00:00,https://dub01.online.tableau.com/#/site/factor...


In [35]:
workbooks_to_refresh = df_workbooks[df_workbooks["tags"].astype(str).str.contains("daily")].id.values

In [37]:
# we expect only one workbook to have a daily refresh
workbooks_to_refresh

array(['91908b3a-d905-4189-a73a-0da53456e5d4'], dtype=object)

In [38]:
# loops through workbooks to refresh and trigger refresh
# run only if you want to refresh the specific dashboard
for w in workbooks_to_refresh:
    workbook = server.workbooks.get_by_id(w)
    # call the refresh method with the data source item
    refreshed_workbook = server.workbooks.refresh(workbook)
    print(f"refresh for workbook id {w} triggered")

refresh for workbook id 91908b3a-d905-4189-a73a-0da53456e5d4 triggered


In [ ]:
# close connections
server.auth.sign_out()